In [2]:
import pandas as pd
import mysql.connector
import requests
from datetime import datetime, timedelta
import numpy as np
import regex as re
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"
from geopy.geocoders import Nominatim
pd.options.display.max_columns = None

In [3]:
geolocator = Nominatim(user_agent="Iris")

# Evaluación II

- Utilizaremos la API de "Universities Hipolabs", una fuente confiable y completa de información sobre las universidades en todo el mundo.

- Nos centraremos en Estados Unidos, Canadá y Argentina.

1. Utilizando la API extraed toda la información que podáis de ella. La url para hacer las llamadas es:


Para esta primera parte de la evaluación, paso por paso, primero definimos en variables los países de los que vamos a querer la información

In [4]:
argentina = 'Argentina'
unitedstates = 'United States'
canada = 'Canada'

Definimos las url por país

In [5]:
can_url = f"http://universities.hipolabs.com/search?country={canada}"
usa_url = f"http://universities.hipolabs.com/search?country={unitedstates}"
arg_url = f"http://universities.hipolabs.com/search?country={argentina}"

### CANADÁ
Llamada a la API y transformación a json.

In [6]:
response_can = requests.get(url=can_url)
response_can.status_code
can_json = response_can.json()

200

In [7]:
df_can = pd.DataFrame(can_json)
df_can.sample(5)

,state-province,domains,name,country,web_pages,alpha_two_code
57,Alberta,[mtroyal.ca],Mount Royal University,Canada,[http://www.mtroyal.ca/],CA
102,Nova Scotia,[ukings.ns.ca],University of King's College,Canada,[http://www.ukings.ns.ca/],CA
134,Ontario,[utoronto.ca],University of Toronto,Canada,[http://www.utoronto.ca/],CA
11,Ontario,[borealc.on.ca],Collège Boréal,Canada,[http://www.borealc.on.ca/],CA
110,New Brunswick,[unbsj.ca],"University of New Brunswick, Saint John",Canada,[http://www.unbsj.ca/],CA


### ESTADOS UNIDOS

Llamada a la API y transformación a json.

In [8]:
response_usa = requests.get(url=usa_url)
response_usa.status_code
usa_json = response_usa.json()

200

In [9]:
df_usa = pd.DataFrame(usa_json)
df_usa.sample(5)

,state-province,domains,name,country,web_pages,alpha_two_code
1881,None,[mcdowelltech.edu],McDowell Technical Community College,United States,[http://www.mcdowelltech.edu],US
1958,None,[mhcc.edu],Mt. Hood Community College,United States,[http://www.mhcc.edu],US
1017,None,[ulm.edu],University of Louisiana at Monroe,United States,[http://www.ulm.edu/],US
1157,None,[vjc.edu],Villa Julie College,United States,[http://www.vjc.edu/],US
1944,None,[redlandscc.edu],Redlands Community College,United States,[http://www.redlandscc.edu],US


### ARGENTINA

Llamada a la API y transformación a json.

In [10]:
response_arg = requests.get(url=arg_url)
response_arg.status_code
arg_json = response_arg.json()

200

In [11]:
df_arg = pd.DataFrame(arg_json)
df_arg.sample(5)

,state-province,domains,name,country,web_pages,alpha_two_code
48,Mendoza,[uncu.edu.ar],Universidad Nacional de Cuyo,Argentina,[http://www.uncu.edu.ar/],AR
68,None,[unnoba.edu.ar],Universidad Nacional del Noroeste de la Provin...,Argentina,[http://www.unnoba.edu.ar/],AR
36,None,[udemm.edu.ar],Universidad de la Marina Mercante,Argentina,[http://www.udemm.edu.ar/],AR
40,Buenos Aires,[uflo.edu.ar],Universidad de Flores - Buenos Aires,Argentina,[http://www.uflo.edu.ar/],AR
75,Buenos Aires,[unsam.edu.ar],Universidad Nacional de General San Martín,Argentina,[http://www.unsam.edu.ar/],AR


2. Una vez tengáis todos los datos de la API, deberéis realizar una serie de procesos de limpieza, estos incluyen:

- Cambiad los nombres de las columnas para homogeneizarlas, tenemos columnas que tienen - y otras _. Unifícalo para que todo vaya con _.

- La columna de domains nos da una información similar a la de web_pages. Eliminad la columna domains.

In [12]:
df = pd.concat([df_usa, df_arg, df_can], axis = 0, join= 'outer') # antes de realizar cambios unimos con .concat los dataframes para realizarlos de una vez

In [13]:
nueva_columna ={col: col.replace("-", "_").lower() for col in df_arg.columns} #list comprehension de cambio de características en las columnas

In [14]:
df.rename(columns = nueva_columna, inplace = True) #renombramos las columnas

In [15]:
df = df.reset_index(drop = True) #eliminamos el índice que se crea solo

In [16]:
df #ya tenemos homogeneizadas las columnas

,state_province,domains,name,country,web_pages,alpha_two_code
0,None,[marywood.edu],Marywood University,United States,[http://www.marywood.edu],US
1,None,[lindenwood.edu],Lindenwood University,United States,[http://www.lindenwood.edu/],US
2,None,[sullivan.edu],Sullivan University,United States,[https://sullivan.edu/],US
3,None,[fscj.edu],Florida State College at Jacksonville,United States,[https://www.fscj.edu/],US
4,None,[xavier.edu],Xavier University,United States,[https://www.xavier.edu/],US
...,...,...,...,...,...,...
2517,Ontario,"[fanshawec.ca, fanshaweonline.ca]",Fanshawe College,Canada,[https://www.fanshawec.ca/],CA
2518,Quebec,[vaniercollege.qc.ca],Vanier College,Canada,[https://www.vaniercollege.qc.ca/],CA
2519,Ontario,"[ontariotechu.ca, ontariotechu.net]",Ontario Tech University,Canada,[https://ontariotechu.ca/],CA
2520,Ontario,[torontomu.ca],Toronto Metropolitan University,Canada,[https://www.torontomu.ca/],CA


In [17]:
df.drop('domains', axis= 1, inplace= True) #eliminamos la columna domains de todo el df

3. Si exploramos la columna de web_pages, nos daremos cuenta que hay universidades, como por ejemplo la Universidad de "Cégep de Saint-Jérôme" de Canadá que en su columna de web_pages tiene más de un valor dentro de la lista. Esto es poco práctico y puede llegar a no tener sentido. el objetivo de este ejericio es que usando el método explode de pandas separéis cada elemento de la lista en una fila nueva. 

In [18]:
df = df.explode('web_pages', ignore_index=False) #.explode() nos permite separar los elementos de esa columna y añadir filas acorde a esas separaciones

In [19]:
df[df['name'] == "Cégep de Saint-Jérôme"] #comprobamos en este ejemplo en concreto si se ha efectuado el .explode(): vemos que sí

,state_province,name,country,web_pages,alpha_two_code
2368,Quebec,Cégep de Saint-Jérôme,Canada,https://www.cstj.qc.ca,CA
2368,Quebec,Cégep de Saint-Jérôme,Canada,https://ccmt.cstj.qc.ca,CA
2368,Quebec,Cégep de Saint-Jérôme,Canada,https://ccml.cstj.qc.ca,CA


4. Una vez hayáis realizado el explode, chequead si tenéis duplicados basándonos unicamente en el nombre de la universidad, en caso de que si, eliminandlos.

In [20]:
df.duplicated('name').sum() #vemos que tenemos 28 duplicados, que corresponden a este explode por filas. Por lo que la universidad se va a repetir, aunque no las url.

28

In [21]:
df[df.duplicated("name")].head() #chequeamos algunos duplicados

,state_province,name,country,web_pages,alpha_two_code
164,None,Augusta University,United States,http://www.gru.edu/,US
231,None,"California State University, Fresno",United States,http://www.fresnostate.edu/,US
249,None,Capella University,United States,http://www.capellauniversity.edu/,US
314,None,Colorado Technical University,United States,http://www.ctuonline.edu/,US
924,None,Thomas Edison State University,United States,http://www.tesu.edu/,US


In [22]:
df_copia = df.copy() #con propósitos de eliminar información, hacemos una copia para perder datos sólo en la copia y no en el df original.

In [23]:
df_copia.drop_duplicates(subset='name', inplace= True) #lo eliminamos desde la copia porque es buena práctica eliminar filas. 

In [24]:
df_copia.duplicated('name').sum() #comprobamos los nulos de la copia y nos da cero

0

In [25]:
df_copia

,state_province,name,country,web_pages,alpha_two_code
0,None,Marywood University,United States,http://www.marywood.edu,US
1,None,Lindenwood University,United States,http://www.lindenwood.edu/,US
2,None,Sullivan University,United States,https://sullivan.edu/,US
3,None,Florida State College at Jacksonville,United States,https://www.fscj.edu/,US
4,None,Xavier University,United States,https://www.xavier.edu/,US
...,...,...,...,...,...
2517,Ontario,Fanshawe College,Canada,https://www.fanshawec.ca/,CA
2518,Quebec,Vanier College,Canada,https://www.vaniercollege.qc.ca/,CA
2519,Ontario,Ontario Tech University,Canada,https://ontariotechu.ca/,CA
2520,Ontario,Toronto Metropolitan University,Canada,https://www.torontomu.ca/,CA


5. Si exploramos la columna de state_province veremos que hay universidades cuyo valor para esta columna es None. Cread una función para reemplazar los None por nulos de numpy.

In [26]:
df[df['state_province'].values == None]  #chequeamos los nones en la fila state_province

,state_province,name,country,web_pages,alpha_two_code
0,None,Marywood University,United States,http://www.marywood.edu,US
1,None,Lindenwood University,United States,http://www.lindenwood.edu/,US
2,None,Sullivan University,United States,https://sullivan.edu/,US
3,None,Florida State College at Jacksonville,United States,https://www.fscj.edu/,US
4,None,Xavier University,United States,https://www.xavier.edu/,US
...,...,...,...,...,...
2457,None,St. Thomas University,Canada,http://www.stthomasu.ca/,CA
2495,None,University of St. Jerome's College,Canada,http://www.usjc.uwaterloo.ca/,CA
2496,None,St. Anne University,Canada,http://www.ustanne.ednet.ns.ca/,CA
2498,None,St. Paul University,Canada,http://www.ustpaul.ca/,CA


In [27]:
def quitar_nones (dataframe): #creamos una pequeña función para eliminar nones

    for col in dataframe:

        dataframe[col] = dataframe[col].fillna(np.nan)

    return dataframe

In [28]:
quitar_nones(df) #funcionó el método .fillna()

,state_province,name,country,web_pages,alpha_two_code
0,NaN,Marywood University,United States,http://www.marywood.edu,US
1,NaN,Lindenwood University,United States,http://www.lindenwood.edu/,US
2,NaN,Sullivan University,United States,https://sullivan.edu/,US
3,NaN,Florida State College at Jacksonville,United States,https://www.fscj.edu/,US
4,NaN,Xavier University,United States,https://www.xavier.edu/,US
...,...,...,...,...,...
2517,Ontario,Fanshawe College,Canada,https://www.fanshawec.ca/,CA
2518,Quebec,Vanier College,Canada,https://www.vaniercollege.qc.ca/,CA
2519,Ontario,Ontario Tech University,Canada,https://ontariotechu.ca/,CA
2520,Ontario,Toronto Metropolitan University,Canada,https://www.torontomu.ca/,CA


6. Después del último cambio, os habréis dado cuenta que tenemos muchos valores nulos dentro de la columna de state_province, por lo que nuestro jefe nos pide que reemplacemos esos nulos por "Unknow". No nos piden ningún método especifico, asi que podremos usar el método que queramos.

In [29]:
def poner_unknowns (dataframe): #función para sustituir NaNs por Unknown. También se puede hacer con el método .rename()

    for col in dataframe:

        dataframe[col] = dataframe[col].fillna('Unknown')

    return dataframe

In [30]:
poner_unknowns(df) #comprobamos

,state_province,name,country,web_pages,alpha_two_code
0,Unknown,Marywood University,United States,http://www.marywood.edu,US
1,Unknown,Lindenwood University,United States,http://www.lindenwood.edu/,US
2,Unknown,Sullivan University,United States,https://sullivan.edu/,US
3,Unknown,Florida State College at Jacksonville,United States,https://www.fscj.edu/,US
4,Unknown,Xavier University,United States,https://www.xavier.edu/,US
...,...,...,...,...,...
2517,Ontario,Fanshawe College,Canada,https://www.fanshawec.ca/,CA
2518,Quebec,Vanier College,Canada,https://www.vaniercollege.qc.ca/,CA
2519,Ontario,Ontario Tech University,Canada,https://ontariotechu.ca/,CA
2520,Ontario,Toronto Metropolitan University,Canada,https://www.torontomu.ca/,CA


7. Ahora nuestros jefes nos piden que saquemos las coordenadas de las provincias donde están ubicadas las universidades. Para eso nos piden que usemos la librería de geopy que aprendimos el día del repaso, leer documentación.  Para desarrollar este ejercicio deberéis:

- Sacar los valores únicos de la columna state_province.

In [31]:
df['state_province'].unique() #así podremos crear un mapa o diccionario para ir sustituyendo datos

array(['Unknown', 'Pennsylvania', 'NV', 'Iowa', 'VA', 'TX', 'Colorado',
       'IN', 'CA', 'South Carolina', 'Washington', 'NY', 'Texas', 'ND',
       'MI', 'Ohio', 'Florida', 'California', 'North Carolina',
       'Michigan', 'GA', 'New York, NY', 'Buenos Aires',
       'Ciudad Autónoma de Buenos Aires', 'Entre Ríos', 'Salta',
       'Córdoba', 'Mendoza', 'Santa Fé', 'Santiago Del Estero',
       'Misiones', 'Catamarca', 'Formosa', 'Jujuy', 'La Rioja',
       'La Pampa', 'San Juan', 'San Luis', 'Tucumán', 'Quebec', 'Ontario',
       'Nova Scotia', 'British Columbia', 'Alberta', 'Manitoba',
       'New Brunswick', 'Saskatchewan', 'Newfoundland and Labrador',
       'Prince Edward Island', 'Yukon'], dtype=object)

In [32]:
dicc = {'NV':'Nevada', 'TX':'Texas', 'IN': 'Indianapolis', 'CA':'California', 'VA':'Virginia', 'NY':'New York','MI':'Michigan', 'GA':'Georgia', 'ND': 'North Dakota', 'Unknown': 'Unknown', 'Pennsylvania':'Pennsylvania','Iowa':'Iowa', 'Colorado':'Colorado','South Carolina':'South Carolina', 'Washington':'Washington', 'Texas':'Texas', 'Ohio':'Ohio','Florida':'Florida', 'California':'California', 'North Carolina':'North Carolina', 'Michigan':'Michigan', 'New York':'New York', 'Buenos Aires':'Buenos Aires',  'Buenos Aires':'Buenos Aires', 'Entre Ríos':'Entre Ríos',  'Salta':'Salta', 'Córdoba':'Córdoba', 'Mendoza':'Mendoza', 'Santa Fé':'Santa Fé',  'Santiago Del Estero':'Santiago Del Estero', 'Misiones':'Misiones','Catamarca':'Catamarca', 'Formosa':'Formosa', 'Jujuy':'Jujuy', 'La Rioja':'La Rioja', 'La Pampa':'La Pampa', 'San Juan':'San Juan', 'San Luis':'San Luis', 'Tucumán':'Tucumán', 'Quebec':'Quebec', 'Ontario':'Ontario','Nova Scotia':'Nova Scotia', 'British Columbia':'British Columbia', 'Alberta':'Alberta', 'Manitoba':'Manitoba', 'New Brunswick':'New Brunswick', 'Saskatchewan':'Saskatchewan', 'Newfoundland and Labrador':'Newfoundland and Labrador', 'Prince Edward Island':'Prince Edward Island', 'Yukon':'Yukon' }
#diccionario para cambiar los valores únicos

In [33]:
df['state_province'] = df['state_province'].map(dicc, na_action= 'ignore') #el .map() nos va a sustituir los valores de la columna por el diccionario que le digamos

In [34]:
df['state_province'].unique()

array(['Unknown', 'Pennsylvania', 'Nevada', 'Iowa', 'Virginia', 'Texas',
       'Colorado', 'Indianapolis', 'California', 'South Carolina',
       'Washington', 'New York', 'North Dakota', 'Michigan', 'Ohio',
       'Florida', 'North Carolina', 'Georgia', nan, 'Buenos Aires',
       'Entre Ríos', 'Salta', 'Córdoba', 'Mendoza', 'Santa Fé',
       'Santiago Del Estero', 'Misiones', 'Catamarca', 'Formosa', 'Jujuy',
       'La Rioja', 'La Pampa', 'San Juan', 'San Luis', 'Tucumán',
       'Quebec', 'Ontario', 'Nova Scotia', 'British Columbia', 'Alberta',
       'Manitoba', 'New Brunswick', 'Saskatchewan',
       'Newfoundland and Labrador', 'Prince Edward Island', 'Yukon'],
      dtype=object)

- Algunos de los valores que tenemos están con siglas, y deberéis reemplazarlos por lo siguiente:

-NV: reemplazalo por Nevada

-TX: reemplazalo por Texas

-IN: reemplazalo por Indianapolis

-CA: reemplazalo por California

-VA: reemplazalo por Virginia

-NY: reemplazalo por New York

-MI: reemplazalo por Michigan

-GA: reemplazalo por Georgia

-ND: reemplazalo por North Dakota

-New York, NY. Deberéis reemplazarlo por "New York".

-'Buenos Aires', 'Ciudad Autónoma de Buenos Aires'. En este caso deberéis poner en ambos casos "Buenos Aires"

- Una vez realizados los pasos anteriores, crea una lista con los valores únicos de las provincias de las universidades.

In [35]:
lista_sitios = list(df['state_province'].unique()) #sacamos la lista de provincias


- Usando la API de geopy, extraed la latitud y la longitud de cada una de las provincias y almacenad los resultados en un dataframe.

In [36]:
latitud = []     #definimos listas vacías para guardar las localizaciones que nos devuelva geopy más tarde
longitud = []
provincia = []

In [37]:
for l in lista_sitios: #bucle para extraer las localizaciones

    location = geolocator.geocode(l)
    provincia.append(l)
    latitud.append(location.latitude)
    longitud.append(location.longitude)

In [38]:
coordenadas = {'latitud':latitud, 'longitud': longitud, 'state_province': provincia} #definición de las columnas

In [39]:
df_coordenada = pd.DataFrame(coordenadas) #pasamos a df el anterior diccionario y así visualizar las columnas

In [40]:
df_coordenada.sample()

,latitud,longitud,state_province
11,40.712728,-74.006015,New York


- Una vez que tengáis los datos del ejercicio anterior en un dataframe, unidlo con el de las universidades que hemos sacado de la API.

In [41]:
df_final = df.merge(df_coordenada, how='inner') #unimos por la columna que tienen en común, "state_province"

In [42]:
df_final

,state_province,name,country,web_pages,alpha_two_code,latitud,longitud
0,Unknown,Marywood University,United States,http://www.marywood.edu,US,25.029422,-77.361956
1,Unknown,Lindenwood University,United States,http://www.lindenwood.edu/,US,25.029422,-77.361956
2,Unknown,Sullivan University,United States,https://sullivan.edu/,US,25.029422,-77.361956
3,Unknown,Florida State College at Jacksonville,United States,https://www.fscj.edu/,US,25.029422,-77.361956
4,Unknown,Xavier University,United States,https://www.xavier.edu/,US,25.029422,-77.361956
...,...,...,...,...,...,...,...
2530,Saskatchewan,University of Regina,Canada,http://www.uregina.ca/,CA,55.532126,-106.141224
2531,Saskatchewan,University of Saskatchewan,Canada,http://www.usask.ca/,CA,55.532126,-106.141224
2532,Newfoundland and Labrador,Memorial University of Newfoundland,Canada,http://www.ucs.mun.ca/,CA,53.821733,-61.229553
2533,Prince Edward Island,University of Prince Edward Island,Canada,http://www.upei.ca/,CA,46.503681,-63.595411


8. Crea una BBDD en mysql que contenga las siguientes tablas:

- Tabla países: donde encontraremos las siguientes columnas:

-idestado: primary key, integer, autoincremental

-nombre_pais: varchar

-nombre_provincia: varchar

-latitud: decimal

-longitud: decimal

- Tabla universidades: donde encontraremos las siguientes columnas:

-iduniversidades: primary key, integer, autoincremental

-nombre_universidad: varchar

-pagina_web: varchar

paises_idestado: foreing key

In [43]:
class Cargar:

    """Esta clase sirve para crear una base de datos y las tablas de la misma. Es necesario introducir el nombre que se desee para la bbdd, la contraseña, 
    y el plugin "mysql_native_password" para aquellos que tengan dificultades con mysql.connector.
    """
    
    def __init__(self, nombre_bbdd, contraseña, plugin):
        
        self.nombre_bbdd = nombre_bbdd
        self.contraseña = contraseña
        self.plugin = plugin

    def crear_bbdd(self):

        mydb = mysql.connector.connect(host="localhost",
                                        user="root",
                                        password=f'{self.contraseña}',
                                        auth_plugin = f'{self.plugin}')
        mycursor = mydb.cursor()

        try:
            mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {self.nombre_bbdd};")
            print(mycursor)
        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)



    def crear_insertar_tabla(self,query):
    
        cnx = mysql.connector.connect(user='root', password=f"{self.contraseña}",
                                        host='127.0.0.1', database=f"{self.nombre_bbdd}", 
                                        auth_plugin = f'{self.plugin}')

        mycursor = cnx.cursor()
        
        try: 
            mycursor.execute(query)
            cnx.commit() 

        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)


In [44]:
unis_mundo = Cargar('universidades_mundo', 'AlumnaAdalab', 'mysql_native_password')

In [45]:
unis_mundo.crear_bbdd()

MySQLCursor: CREATE DATABASE IF NOT EXISTS universida..


In [46]:
query_paises = """ CREATE TABLE IF NOT EXISTS `universidades_mundo`.`paises` (
  `idestado` INT NOT NULL,
  `nombre_pais` VARCHAR(45) NOT NULL,
  `nombre_provincia` VARCHAR(45) NOT NULL,
  `latitud` DECIMAL NOT NULL,
  `longitud` VARCHAR(45) NOT NULL,
  PRIMARY KEY (`idestado`))
ENGINE = InnoDB; """ 

In [47]:
query_universidades = """ CREATE TABLE IF NOT EXISTS `universidades_mundo`.`universidades` (
  `iduniversidades` INT NOT NULL AUTO_INCREMENT,
  `nombre_universidad` VARCHAR(45) NOT NULL,
  `pagina_web` VARCHAR(100) NOT NULL,
  `paises_idestado` INT NOT NULL,
  PRIMARY KEY (`iduniversidades`),
  INDEX `fk_universidades_paises1_idx` (`paises_idestado` ASC) VISIBLE,
  CONSTRAINT `fk_universidades_paises1`
    FOREIGN KEY (`paises_idestado`)
    REFERENCES `universidades_mundo`.`paises` (`idestado`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION)
ENGINE = InnoDB;"""

In [48]:
unis_mundo.crear_insertar_tabla(query_paises)
unis_mundo.crear_insertar_tabla(query_universidades)

9. Introduce todo el código que habéis ido creando en funciones, siguiendo la misma lógica que hemos seguido en los pairs

In [49]:
def llamar_API_2(paises): #función que recoge de la API los datos correspondientes a los países introducidos en el parámetro

    df_universidades = pd.DataFrame()

    for pais in paises:

        url = f"http://universities.hipolabs.com/search?country={pais}"
        response = requests.get(url=url)
        datos_json = pd.json_normalize(response.json())
        df_universidades = pd.concat([df_universidades, datos_json], axis= 0, ignore_index= True)

    return df_universidades
    

In [50]:
df_api2 = llamar_API_2(['Canada', 'United States', 'Argentina']) #ejecutamos la función

In [51]:
df_api2.sample(5) #comprobamos

,state-province,domains,name,country,web_pages,alpha_two_code
873,None,[pwu.com],Pacific Western University,United States,[http://www.pwu.com/],US
649,None,[iupui.edu],Indiana University/Purdue University at Indian...,United States,[http://www.iupui.edu/],US
1189,None,[miami.edu],University of Miami,United States,[http://welcome.miami.edu/],US
131,Ontario,[usudbury.com],University of Sudbury,Canada,[http://www.usudbury.com/],CA
328,None,[bsu.edu],Ball State University,United States,[http://www.bsu.edu/],US


In [52]:
def limpiar_df(dataframe, columnas):  #función para homogeneizar columnas y eliminar las que nos han pedido

    nuevas_columnas = {col: col.replace("-", "_") for col in dataframe.columns}
    dataframe.rename(columns=nuevas_columnas, inplace= True)
    dataframe.drop(columnas, axis =1, inplace= True)
    return dataframe

In [53]:
df_api2 = limpiar_df(df_api2, 'domains') #llamamos a la función

In [54]:
df_api2.sample(5) #comprobamos si ha funcionado

,state_province,name,country,web_pages,alpha_two_code
1796,None,Greenfield Community College,United States,[http://www.gcc.mass.edu],US
1959,None,American Academy of Dramatic Arts-​New York,United States,[http://www.aada.edu],US
2395,None,Interdenominational Theological Center,United States,[https://www.itc.edu/],US
2180,None,Cedar Valley College,United States,[http://www.cedarvalleycollege.edu],US
1410,None,George C Wallace State Community College-​Dothan,United States,[http://www.wallace.edu],US


In [55]:
def explode_metodo(dataframe, columna): #función para ampliar las filas acorde a las urls, aumentando las filas del df según tantas urls tengamos

    dataframe = df.explode(columna, ignore_index=False)
    
    return dataframe

In [56]:
df_api2.sample(8)

,state_province,name,country,web_pages,alpha_two_code
2412,None,Paul Quinn College,United States,[https://paulquinn.edu/],US
2229,None,Tyler Junior College,United States,[http://WWW.TJC.EDU],US
823,None,North American University,United States,[http://www.na.edu/],US
2233,None,Western Texas College,United States,[http://www.wtc.edu],US
2226,None,Texas State Technical College-​Waco,United States,[http://www.waco.tstc.edu],US
2341,Pennsylvania,Point Park University,United States,[https://pointpark.edu/],US
505,None,DePauw University,United States,[http://www.depauw.edu/],US
1883,None,Southwest Mississippi Community College,United States,[http://www.smcc.edu],US


In [57]:
df_api2 = explode_metodo(df_api2, 'web_pages') #llamamos a la función y a la columna que le queremos hacer el método .explode()

In [58]:
df_api2[df_api2['name'] == "Cégep de Saint-Jérôme"] #comprobamos si ha funcionado el .explode()

,state_province,name,country,web_pages,alpha_two_code
2368,Quebec,Cégep de Saint-Jérôme,Canada,https://www.cstj.qc.ca,CA
2368,Quebec,Cégep de Saint-Jérôme,Canada,https://ccmt.cstj.qc.ca,CA
2368,Quebec,Cégep de Saint-Jérôme,Canada,https://ccml.cstj.qc.ca,CA


In [59]:
def quitar_nones2 (dataframe): 

    for col in dataframe:
            
            dataframe[col] = dataframe[col].replace(np.nan)

    return dataframe

In [60]:
df_api2 = quitar_nones2(df_api2)
df_api2.sample()

,state_province,name,country,web_pages,alpha_two_code
467,Unknown,Hesston College,United States,http://www.hesston.edu/,US


In [61]:
def poner_unknowns2 (dataframe):

    for col in dataframe:

        dataframe[col] = dataframe[col].fillna('Unknown')

    return dataframe

In [62]:
df_api2 = poner_unknowns2(df_api2)

In [63]:
df_api2.sample(5)

,state_province,name,country,web_pages,alpha_two_code
1245,Unknown,Youngstown State University,United States,http://www.ysu.edu/,US
2274,Unknown,Tougaloo College,United States,https://www.tougaloo.edu/,US
2442,Ontario,Royal Military College of Canada,Canada,http://www.rmc.ca/,CA
1758,Unknown,Mid-​Plains Community College,United States,http://www.mpcc.edu,US
906,Unknown,Temple University,United States,http://www.temple.edu/,US


In [64]:
def buscar_lugares(dataframe, columna, diccionario): #buscamos las ubicaciones de los lugares contenidos en el df

    latitud = []
    longitud = []
    provincia = []

    dataframe[columna] = dataframe[columna].map(diccionario, na_action="ignore")  #sustituirá cada valor de la columna por otros que le introduzcamos
    lista_provincias = list(dataframe[columna].unique())
    geolocator = Nominatim(user_agent = "Iris")
    for i in lista_provincias:
        location = geolocator.geocode(i)
        provincia.append(i)
        latitud.append(location.latitude)
        longitud.append(location.longitude)

    coordenadas2 = {"state_province":provincia,
                   "latitud":latitud,
                   "longitud":longitud}
    dataframe_coordenada2 = pd.DataFrame(coordenadas2)
    dataframe_final2 =  dataframe.merge(dataframe_coordenada2, on = columna)

    return dataframe_final2

In [65]:
df_api2 = buscar_lugares(df_api2, 'state_province', dicc) #llamamos a la función

In [66]:
df_api2 #comprobamos

,state_province,name,country,web_pages,alpha_two_code,latitud,longitud
0,Unknown,Marywood University,United States,http://www.marywood.edu,US,25.029422,-77.361956
1,Unknown,Lindenwood University,United States,http://www.lindenwood.edu/,US,25.029422,-77.361956
2,Unknown,Sullivan University,United States,https://sullivan.edu/,US,25.029422,-77.361956
3,Unknown,Florida State College at Jacksonville,United States,https://www.fscj.edu/,US,25.029422,-77.361956
4,Unknown,Xavier University,United States,https://www.xavier.edu/,US,25.029422,-77.361956
...,...,...,...,...,...,...,...
2530,Saskatchewan,University of Regina,Canada,http://www.uregina.ca/,CA,55.532126,-106.141224
2531,Saskatchewan,University of Saskatchewan,Canada,http://www.usask.ca/,CA,55.532126,-106.141224
2532,Newfoundland and Labrador,Memorial University of Newfoundland,Canada,http://www.ucs.mun.ca/,CA,53.821733,-61.229553
2533,Prince Edward Island,University of Prince Edward Island,Canada,http://www.upei.ca/,CA,46.503681,-63.595411


10. BONUS

- Introduce los datos en la BBDD de SQL.

- Crea una clase con todo el código generado en esta evaluación.

In [67]:
class Extraer:

    """Esta clase sirve para extraer datos e importarlos a Pandas desde la API "hipolabs". Procesos: Hace la llamada a la API y guarda en un dataframe los datos; elimina las columnas
    innecesarias (para esta evaluación) y hace homogéneas sus columnas; separa las urls de cada universidad para contenerlas todas; sustituye los nones por nulos de numpy y los
    sustituye por "Unknown"; busca los lugares de los países y añade sus coordenadas al dataframe final.
    El parámetro requerido son los países que se quieran insertar, en este caso serán Argentina, Canadá y Estados Unidos
    """

    def __init__(self, paises):

        self.paises = paises

    def llamar_API_unis(self):

        df_universidades = pd.DataFrame()

        for pais in self.paises:

            url = f"http://universities.hipolabs.com/search?country={pais}"
            response = requests.get(url=url)
            datos_json = pd.json_normalize(response.json())
            df_universidades = pd.concat([df_universidades, datos_json], axis= 0, ignore_index= True)

        return df_universidades
    
    def limpiar_df(self, dataframe, columnas):

        self.dataframe = dataframe
        self.columnas = columnas

        nuevas_columnas = {col: col.replace("-", "_") for col in dataframe.columns}
        dataframe.rename(columns=nuevas_columnas, inplace= True)
        dataframe.drop(columnas, axis =1, inplace= True)
        
        return dataframe
    
    
    def explode_metodo(self, dataframe, columna):

        dataframe = df.explode(columna, ignore_index=False)
        
        return dataframe
    
    def quitar_nones(self, dataframe):

        for col in dataframe:
                
                dataframe[col] = dataframe[col].replace(np.nan)

        return dataframe
    

    def poner_unknowns(self, dataframe):

        for col in dataframe:

            dataframe[col] = dataframe[col].fillna('Unknown')

        return dataframe
    
    def buscar_lugares(self, dataframe, columna, diccionario, user):

        self.diccionario = diccionario
        self.user = user

        latitud = []
        longitud = []
        provincia = []

        dataframe[columna] = dataframe[columna].map(diccionario, na_action="ignore")
        lista_provincias = list(dataframe[columna].unique())
        geolocator = Nominatim(user_agent = user)

        for i in lista_provincias:
            location = geolocator.geocode(i)
            provincia.append(i)
            latitud.append(location.latitude)
            longitud.append(location.longitude)

        coordenadas = {columna :provincia,
                    "latitud":latitud,
                    "longitud":longitud}
        dataframe_coordenada = pd.DataFrame(coordenadas)
        dataframe_final =  dataframe.merge(dataframe_coordenada, on = columna)

        return dataframe_final

In [68]:
mundo = Extraer(['Argentina', 'Canada', 'United States']) #instanciamos la clase con los parámetros que necesitamos

In [69]:
df_universidades1 = mundo.llamar_API_unis() #con la clase extraemos los datos

In [70]:
df_universidades1

,state-province,domains,name,country,web_pages,alpha_two_code
0,Buenos Aires,[atlantida.edu.ar],Universidad Atlantida Argentina,Argentina,[http://www.atlantida.edu.ar/],AR
1,Buenos Aires,[austral.edu.ar],Universidad Austral Buenos Aires,Argentina,[http://www.austral.edu.ar/],AR
2,Ciudad Autónoma de Buenos Aires,[caece.edu.ar],"Universidad CAECE, Buenos Aires",Argentina,[http://www.caece.edu.ar/],AR
3,Ciudad Autónoma de Buenos Aires,[cema.edu.ar],Instituto Universitario CEMA,Argentina,[http://www.cema.edu.ar/],AR
4,Ciudad Autónoma de Buenos Aires,[iese.edu.ar],Instituto de Enseñanza Superior del Ejército,Argentina,[http://www.iese.edu.ar/],AR
...,...,...,...,...,...,...
2517,None,[vul.edu],Virginia University of Lynchburg,United States,[https://www.vul.edu/],US
2518,None,[voorhees.edu],Voorhees University,United States,[https://www.voorhees.edu/],US
2519,None,[wvstate.edu],West Virginia State University,United States,[https://www.wvstateu.edu/],US
2520,None,[wileyc.edu],Wiley College,United States,[https://www.wileyc.edu/],US


In [71]:
df_universidades1 = mundo.limpiar_df(df_universidades1, 'domains') #limpiamos columnas que no queremos

In [72]:
df_universidades1.sample()

,state_province,name,country,web_pages,alpha_two_code
1302,None,University of North Carolina at Charlotte,United States,[http://www.uncc.edu/],US


In [73]:
df_universidades1 = mundo.explode_metodo(df_universidades1, 'web_pages')

In [74]:
df_universidades1 = mundo.quitar_nones(df_universidades1)

In [75]:
df_universidades1 = mundo.poner_unknowns(df_universidades1)

In [76]:
df_universidades1 = mundo.buscar_lugares(df_universidades1, 'state_province', dicc, 'Iris') #para extraer coordenadas le decimos dónde las tiene que ubicar, de qué columna extraer
#las localizaciones, el diccionario con los lugares, y el user para poder usar geopy

In [77]:
df_universidades1 #comprobamos si ha funcionado con la clase

,state_province,name,country,web_pages,alpha_two_code,latitud,longitud
0,Unknown,Marywood University,United States,http://www.marywood.edu,US,25.029422,-77.361956
1,Unknown,Lindenwood University,United States,http://www.lindenwood.edu/,US,25.029422,-77.361956
2,Unknown,Sullivan University,United States,https://sullivan.edu/,US,25.029422,-77.361956
3,Unknown,Florida State College at Jacksonville,United States,https://www.fscj.edu/,US,25.029422,-77.361956
4,Unknown,Xavier University,United States,https://www.xavier.edu/,US,25.029422,-77.361956
...,...,...,...,...,...,...,...
2530,Saskatchewan,University of Regina,Canada,http://www.uregina.ca/,CA,55.532126,-106.141224
2531,Saskatchewan,University of Saskatchewan,Canada,http://www.usask.ca/,CA,55.532126,-106.141224
2532,Newfoundland and Labrador,Memorial University of Newfoundland,Canada,http://www.ucs.mun.ca/,CA,53.821733,-61.229553
2533,Prince Edward Island,University of Prince Edward Island,Canada,http://www.upei.ca/,CA,46.503681,-63.595411
